In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [ ]:
import os
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [ ]:
# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
# Creating Agents
# Agent 1 - Venue Coordinator
venue_coordinator_agent = Agent(
    role = "Venue Coordinator",
    goal = "Search for an appropriate venue based on event requirements",
    backstory = "We are orgainising a meetup of closed group of people who meet inside a restaurant/cafe and discuss stuffs "
                " related to GenAI ",
    tools = [search_tool, scrape_tool],
    verbose = True,
    allow_delegation = False,
)

# Agent 2 - Date-Venue Mapper
date_venue_mapper_agent = Agent(
    role = "Date Venue Mapper",
    goal = "Allocate venue for all future meetups starting tomorrow",
    backstory = "We meet every Sunday for GenAI discussions starting tomorrow "
                " and plan on getting venue decided now only "
                " for all future meetups coming in next {month_count} months. "
                " Get today's date from internet (double check today's date from mutiple sources) "
                " and don't show any past dates ",
    tools = [search_tool],
    verbose = True,
    allow_delegation = False,
)

In [ ]:
# Creating Tasks
# Task 1 - Venue Locator
venue_locator_task = Task(
    description = "Search for at least 10 cafes or restaurant in {location} "
                   " which can accommodate at least {headcount} people. "
                    " Make sure that the cafe serves really good coffee. Bonus points if it serves speciality coffee. "
                    " Also it should be highly rated on zomato and swiggy both "
                    " Choose only those cafes which are known to be quiet and ideal for discussions. "
                    " Avoid places which serve alcohol "
                    " Priotize those cafes which lie in the center of {location} ",
    expected_output = "List of venues with complete address, opening/closing timings, most recommended item, "
                        " rating on zomato, rating on swiggy, distance from millenium city metro station",
    #human_input = True,
    agent = venue_coordinator_agent,
)

# Task 2 - Date - Venue Mapping
date_venue_mapping_task = Task(
    description = " Based on the top cafe/restaurant recommendations of venue_locator_task "
                    " plan the venues for meetup which happens every Sunday for next {month_count} months "
                    " Take start date as tomorrow and allocate one venue for each future meetup "
                    " but ensure that the venue doesn't repeat ",
    expected_output = " List of future Sundays with date for next {month_count} months "
                        " and the venue allocated to each meetup "
                        " with complete address, opening/closing timings, most recommended item, "
                        " rating on zomato, rating on swiggy, distance from millenium city metro station ",
    agent = date_venue_mapper_agent,
    human_input = True,
)

In [ ]:
venue_hunter_crew = Crew(
    agents = [venue_coordinator_agent, date_venue_mapper_agent],
    tasks = [venue_locator_task, date_venue_mapping_task],
    verbose = True,
)

In [ ]:
event_details = {
    "location": ["Gurgaon, India"],
    "headcount": 10,
    "month_count" : 3,
}

In [ ]:
result = venue_hunter_crew.kickoff(inputs=event_details)